In [ ]:
import cv2
import numpy as np
import face_recognition
import os
import datetime
import pandas as pd
import json


In [ ]:
path='Images'
images=[]
personName=[]
myList=os.listdir(path)
print(myList)
for cu_img in myList:
    current_Img=cv2.imread(f'{path}/{cu_img}')
    images.append(current_Img)
    personName.append(os.path.splitext(cu_img)[0])
print(personName)
#print(images)

In [3]:
def faceEncodings(images):
    encodeList=[]
    for img in images:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode=face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList
encodeListKnown=faceEncodings(images)

In [5]:
#data=pd.read_csv('attendance.csv')
#data`
#data.get(['Name','Date'])

In [4]:

def attendance(name,date):
    with open('attendance.csv','r+') as f:
        myDataList=f.readlines()
        nameList=[]
        dateList=[]
        
        for line in myDataList:
            entry=line.split(',')
            nameList.append(entry[0])
            dateList.append(entry[1])
            #timeList.append(entry[2])
            
            
        if (date,name) not in zip(dateList,nameList):        
            #time_now=datetime.now()
            #tStr=time_now.strftime('%H:%M:%S')
            
            def time_in_range(start, end, entry_time):
                """Returns whether current is in the range [start, end]"""
                return start <= entry_time <= end
            
            start = datetime.time(9,15, 0)
            end = datetime.time(18, 0, 0)
            entry_time = datetime.datetime.now().time()
            #time_in_range(start, end, current)
            
            
            if time_in_range(start, end, entry_time):
                with open('sample.json', 'r') as openfile:
                    json_object = json.load(openfile) #load function convert json object to python dictionary 
                    
                    f.writelines(f'\n{name},{date},{json_object[str(name)]},{entry_time}')
                        
                #print(current)
                #f.writelines(f'\n{name},{date},{entry_time}')
                
        data=pd.read_csv('attendance.csv')
        
        #if (name,date) in zip(nameList,dateList):
            
        def time_out_range(start, end, leave_time):   
            """Returns whether current is in the range [start, end]"""
            return start <= leave_time <= end
        
        start = datetime.time(18,30, 0)
        end = datetime.time(23, 59, 0)
        leave_time = datetime.datetime.now().time()
                    
            
        if time_out_range(start, end, leave_time):
            filter1 = data["Name"]==name
            filter2 = data["Date"]==date
            data['Leave-Time']= np.where((filter1 & filter2),leave_time,data['Leave-Time'])
            data.to_csv('attendance.csv',index=False)
        


In [6]:

cap=cv2.VideoCapture(0)

while True:
    ret,frame=cap.read()
    faces=cv2.resize(frame,(0,0),None,0.25,0.25)
    faces=cv2.cvtColor(faces,cv2.COLOR_BGR2RGB)
    
    facesCurrentFrame=face_recognition.face_locations(faces)
    encodesCurrentFrame=face_recognition.face_encodings(faces,facesCurrentFrame)
    
    for encodeFace,faceLoc in zip(encodesCurrentFrame,facesCurrentFrame):
        matches=face_recognition.compare_faces(encodeListKnown,encodeFace)
        faceDis=face_recognition.face_distance(encodeListKnown,encodeFace)
        
        matchIndex =np.argmin(faceDis)
        
        if matches[matchIndex]:
            name=personName[matchIndex]
            #print(name)
            y1,x2,y2,x1=faceLoc
            y1,x2,y2,x1=y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(frame,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(frame,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
            
            time_now=datetime.datetime.now() 
            date=time_now.strftime('%d/%m/%Y')
            attendance(name,date)
            
    cv2.imshow('Camera',frame)
    # speak(f"Hello {name}")
    
    if cv2.waitKey(10)==13:
       
        break
    

print(matchIndex)        
cap.release()
cv2.destroyAllWindows()



4
